# import

In [1]:
# import packages and libraries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

# Data cleaning

In [2]:
df = pd.read_csv("rents.csv")

In [3]:
df

,location,price,beds
0,austin-tx,"$2,549 - 7,938",Studio - 3 Beds
1,austin-tx,"$2,025 - 13,235",Studio - 3 Beds
2,austin-tx,"$1,785 - 2,565",1-2 Beds
3,austin-tx,"$1,595 - 2,303",1-2 Beds
4,austin-tx,"$1,600 - 2,175",2-3 Beds
...,...,...,...
870,new-york-ny,NaN,NaN
871,new-york-ny,NaN,NaN
872,new-york-ny,NaN,NaN
873,new-york-ny,NaN,NaN


In [4]:
# drop empty rows
df = df.dropna()
df

,location,price,beds
0,austin-tx,"$2,549 - 7,938",Studio - 3 Beds
1,austin-tx,"$2,025 - 13,235",Studio - 3 Beds
2,austin-tx,"$1,785 - 2,565",1-2 Beds
3,austin-tx,"$1,595 - 2,303",1-2 Beds
4,austin-tx,"$1,600 - 2,175",2-3 Beds
...,...,...,...
821,new-york-ny,"$3,550",2 Beds
822,new-york-ny,"$5,000",4 Beds
823,new-york-ny,"$5,500",4 Beds
824,new-york-ny,"$4,300",3 Beds


## Transform column "price"

In [5]:
#remove the dollar sign
df["price"] = df["price"].str.extract("\$(.*)")

# remove delimiter comma in the price column
df["price"] = df["price"].str.replace(",", "")

#extract the prices before the dash delimiter"-" and after the dash
# and concatenate them to the dataframe
df = pd.concat([df, df["price"].str.split("-", expand=True)], axis=1)

# rename the column names
df.rename(columns={0: "price_1", 1: "price_2"}, inplace=True)

# remove the column price
df.drop("price", inplace=True, axis=1)

C:\Users\Tuyen\AppData\Local\Temp/ipykernel_2960/2155628185.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price"] = df["price"].str.extract("\$(.*)")
C:\Users\Tuyen\AppData\Local\Temp/ipykernel_2960/2155628185.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price"] = df["price"].str.replace(",", "")


In [6]:
df.head()
df

,location,beds,price_1,price_2
0,austin-tx,Studio - 3 Beds,2549,7938
1,austin-tx,Studio - 3 Beds,2025,13235
2,austin-tx,1-2 Beds,1785,2565
3,austin-tx,1-2 Beds,1595,2303
4,austin-tx,2-3 Beds,1600,2175
...,...,...,...,...
821,new-york-ny,2 Beds,3550,None
822,new-york-ny,4 Beds,5000,None
823,new-york-ny,4 Beds,5500,None
824,new-york-ny,3 Beds,4300,None


## Transform column "beds"

In [7]:
# change from Studio to 1
df["beds"] = df["beds"].apply(lambda entry: re.sub("Studio", "1", entry) if "Studio" in entry else entry)

# remove the "Beds" word
df["beds"] = df["beds"].str.replace(" Beds", "")
df["beds"] = df["beds"].str.replace(" Bed", "")

In [8]:
np.sum(df["beds"].isnull())

0

In [9]:
df

,location,beds,price_1,price_2
0,austin-tx,1 - 3,2549,7938
1,austin-tx,1 - 3,2025,13235
2,austin-tx,1-2,1785,2565
3,austin-tx,1-2,1595,2303
4,austin-tx,2-3,1600,2175
...,...,...,...,...
821,new-york-ny,2,3550,None
822,new-york-ny,4,5000,None
823,new-york-ny,4,5500,None
824,new-york-ny,3,4300,None


In [10]:
# get the number of beds before "-" after dash
df["beds_1"] = df["beds"].str.extract("(\d).*")
df["beds_2"] = df["beds"].str.extract(".*-.*(\d)")

# drop the column beds
df.drop("beds", axis=1, inplace=True)

In [11]:
df

,location,price_1,price_2,beds_1,beds_2
0,austin-tx,2549,7938,1,3
1,austin-tx,2025,13235,1,3
2,austin-tx,1785,2565,1,2
3,austin-tx,1595,2303,1,2
4,austin-tx,1600,2175,2,3
...,...,...,...,...,...
821,new-york-ny,3550,None,2,NaN
822,new-york-ny,5000,None,4,NaN
823,new-york-ny,5500,None,4,NaN
824,new-york-ny,4300,None,3,NaN


## Merging data

In [12]:
newdf = pd.DataFrame()

In [13]:
# concat the first part of the rents (price_1, beds_1) to our new df
first_part = df.loc[:, ["location", "price_1", "beds_1"]].rename(columns={"price_1": "price", "beds_1": "beds"}).dropna()
newdf = pd.DataFrame(first_part)
newdf

,location,price,beds
0,austin-tx,2549,1
1,austin-tx,2025,1
2,austin-tx,1785,1
3,austin-tx,1595,1
4,austin-tx,1600,2
...,...,...,...
821,new-york-ny,3550,2
822,new-york-ny,5000,4
823,new-york-ny,5500,4
824,new-york-ny,4300,3


In [14]:
# concat the second part of the rents (price_2, beds_2) to our new df
second_part = df.loc[:, ["location", "price_2", "beds_2"]].rename(columns={"price_2": "price", "beds_2":"beds"}).dropna()
newdf = pd.concat([newdf, second_part], axis=0, ignore_index=True)
newdf

,location,price,beds
0,austin-tx,2549,1
1,austin-tx,2025,1
2,austin-tx,1785,1
3,austin-tx,1595,1
4,austin-tx,1600,2
...,...,...,...
1055,new-york-ny,23015,3
1056,new-york-ny,16190,4
1057,new-york-ny,7046,2
1058,new-york-ny,3010,1


In [15]:
# drop nan records
newdf.dropna(inplace=True)

In [16]:
# convert data types of price and beds to int
newdf["price"]=newdf["price"].astype("int")
newdf["beds"]=newdf["beds"].astype("int")

In [17]:
# DATA AFTER BEING CLEANED
newdf

,location,price,beds
0,austin-tx,2549,1
1,austin-tx,2025,1
2,austin-tx,1785,1
3,austin-tx,1595,1
4,austin-tx,1600,2
...,...,...,...
1055,new-york-ny,23015,3
1056,new-york-ny,16190,4
1057,new-york-ny,7046,2
1058,new-york-ny,3010,1


In [18]:
newdf.to_csv("cleaned_rents.csv")